In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
import ast
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
big_df = pd.read_parquet("hf://datasets/wandb/RAGTruth-processed/" + splits["test"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
big_df = big_df[big_df['quality'] == 'good']

In [ ]:
def extract_binary_label(hallucination_labels):
    return 1 if any(value == 1 for value in hallucination_labels.values()) else 0
big_df['binary_label'] = big_df['hallucination_labels_processed'].apply(extract_binary_label)

In [ ]:
summary_df = big_df[big_df['task_type'] == 'Summary']

true_examples = summary_df[summary_df['binary_label'] == True].sample(n=50, random_state=42)
false_examples = summary_df[summary_df['binary_label'] == False].sample(n=50, random_state=42)

balanced_df = pd.concat([true_examples, false_examples]).sample(frac=1, random_state=42)

balanced_df.to_csv("balanced_summary_dataset.csv", index=False)

In [ ]:
small_df = pd.read_json("hf://datasets/aporia-ai/rag_hallucinations/rag_hallucinations_dataset.json")
small_df['binary_label'] = small_df['is_hallucination'].map({True: 1, False: 0})
small_df.rename(columns={"question": "query", "answer": "output"}, inplace=True)

In [ ]:
small_df = small_df.sample(frac=1, random_state=42).reset_index(drop=True)

false_samples = small_df[small_df["binary_label"] == 0].head(100)
true_samples = small_df[small_df["binary_label"] == 1].head(100)

balanced_small_test = pd.concat([false_samples, true_samples]).sample(frac=1, random_state=42).reset_index(drop=True)
balanced_small_test.to_csv("/content/balanced_small_test.csv")

In [7]:
small_balanced_df = pd.read_csv("/content/balanced_small_test.csv")

In [8]:
summary_dataset = pd.read_csv("/content/balanced_summary_dataset.csv")

In [10]:
def tokenize_function(row, tokenizer, max_length=512):
    query_max_len = max_length // 4
    context_max_len = max_length // 2
    output_max_len = max_length // 4

    query_text = f"QUERY: {row['query']} "
    context_text = f"CONTEXT: {row['context']} "
    output_text = f"OUTPUT: {row['output']} "

    query_tokens = tokenizer(query_text, truncation=True, padding='max_length', max_length=query_max_len, return_tensors="pt")
    context_tokens = tokenizer(context_text, truncation=True, padding='max_length', max_length=context_max_len, return_tensors="pt")
    output_tokens = tokenizer(output_text, truncation=True, padding='max_length', max_length=output_max_len, return_tensors="pt")

    combined_input_ids = torch.cat([
        query_tokens['input_ids'][0],
        context_tokens['input_ids'][0],
        output_tokens['input_ids'][0]
    ]).unsqueeze(0)

    combined_attention_mask = torch.cat([
        query_tokens['attention_mask'][0],
        context_tokens['attention_mask'][0],
        output_tokens['attention_mask'][0]
    ]).unsqueeze(0)

    return {"input_ids": combined_input_ids, "attention_mask": combined_attention_mask}

In [11]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):

        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        tokens = tokenize_function(row, self.tokenizer, self.max_length)
        labels = {
            "binary_label": torch.tensor(row['binary_label'], dtype=torch.float),
        }
        return {**tokens, **labels}

In [20]:
from transformers import AutoTokenizer
import torch

def collate_fn(batch):

    model_name = "microsoft/deberta-v3-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

    input_ids = []
    attention_masks = []
    binary_labels = []

    # Iterate over the examples in the batch and prepare the data
    for example in batch:
        # Ensure each sequence has the correct shape (1D tensor)
        input_ids.append(example['input_ids'].squeeze(0))  # Remove the batch dimension
        attention_masks.append(example['attention_mask'].squeeze(0))
        binary_labels.append(example['binary_label'])


    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_masks = torch.nn.utils.rnn.pad_sequence(attention_masks, batch_first=True, padding_value=0)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'binary_label':  torch.tensor(binary_labels, dtype=torch.float)
    }

In [13]:
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

def test_model(model, test_dataset, tokenizer, batch_size, device):
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

    model.to(device)
    model.eval()

    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['binary_label'].to(device).long()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            probabilities = torch.sigmoid(logits)
            predictions = (probabilities >= 0.5).long()

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    report = classification_report(all_labels, all_predictions, target_names=["Class 0", "Class 1"], zero_division=1)
    accuracy = accuracy_score(all_labels, all_predictions)

    print("Classification Report:")
    print(report)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    return report, accuracy




### Testing bert-base

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_bin = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
model_bin.load_state_dict(torch.load("/content/drive/MyDrive/hallucinations/model_bert_5_epochs.pth"))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
balanced_small_dataset = CustomDataset(small_balanced_df, tokenizer, max_length = 512)

In [ ]:
summary_test_dataset = CustomDataset(summary_dataset, tokenizer, max_length = 512)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 32
test_model(model_bin, balanced_small_dataset, tokenizer, batch_size, device)

Evaluating: 100%|██████████| 7/7 [00:09<00:00,  1.33s/it]

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.60      0.94      0.73       100
     Class 1       0.86      0.37      0.52       100

    accuracy                           0.66       200
   macro avg       0.73      0.66      0.62       200
weighted avg       0.73      0.66      0.62       200

Accuracy: 65.50%


('              precision    recall  f1-score   support\n\n     Class 0       0.60      0.94      0.73       100\n     Class 1       0.86      0.37      0.52       100\n\n    accuracy                           0.66       200\n   macro avg       0.73      0.66      0.62       200\nweighted avg       0.73      0.66      0.62       200\n',
 0.655)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 32
test_model(model_bin, summary_test_dataset, tokenizer, batch_size, device)

Evaluating: 100%|██████████| 4/4 [00:04<00:00,  1.22s/it]

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.56      0.82      0.67        50
     Class 1       0.67      0.36      0.47        50

    accuracy                           0.59       100
   macro avg       0.61      0.59      0.57       100
weighted avg       0.61      0.59      0.57       100

Accuracy: 59.00%


('              precision    recall  f1-score   support\n\n     Class 0       0.56      0.82      0.67        50\n     Class 1       0.67      0.36      0.47        50\n\n    accuracy                           0.59       100\n   macro avg       0.61      0.59      0.57       100\nweighted avg       0.61      0.59      0.57       100\n',
 0.59)

### testing roberta-base

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
model.load_state_dict(torch.load("/content/drive/MyDrive/hallucinations/model_roberta_5_epochs.pth",  map_location=torch.device('cpu')))


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
balanced_small_dataset = CustomDataset(small_balanced_df, tokenizer, max_length = 512)

In [ ]:
summary_test_dataset = CustomDataset(summary_dataset, tokenizer, max_length = 512)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 32
test_model(model, balanced_small_dataset, tokenizer, batch_size, device)

Evaluating: 100%|██████████| 7/7 [00:07<00:00,  1.03s/it]

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.96      0.92      0.94       100
     Class 1       0.92      0.96      0.94       100

    accuracy                           0.94       200
   macro avg       0.94      0.94      0.94       200
weighted avg       0.94      0.94      0.94       200

Accuracy: 94.00%


('              precision    recall  f1-score   support\n\n     Class 0       0.96      0.92      0.94       100\n     Class 1       0.92      0.96      0.94       100\n\n    accuracy                           0.94       200\n   macro avg       0.94      0.94      0.94       200\nweighted avg       0.94      0.94      0.94       200\n',
 0.94)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 32
test_model(model, summary_test_dataset, tokenizer, batch_size, device)

Evaluating: 100%|██████████| 4/4 [00:03<00:00,  1.01it/s]

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.60      0.68      0.64        50
     Class 1       0.63      0.54      0.58        50

    accuracy                           0.61       100
   macro avg       0.61      0.61      0.61       100
weighted avg       0.61      0.61      0.61       100

Accuracy: 61.00%


('              precision    recall  f1-score   support\n\n     Class 0       0.60      0.68      0.64        50\n     Class 1       0.63      0.54      0.58        50\n\n    accuracy                           0.61       100\n   macro avg       0.61      0.61      0.61       100\nweighted avg       0.61      0.61      0.61       100\n',
 0.61)

## Testing DEBERTA

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model_bin = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
model_bin.load_state_dict(torch.load("/content/drive/MyDrive/hallucinations/model_deberta_5_epochs.pth"))


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [14]:
balanced_small_dataset = CustomDataset(small_balanced_df, tokenizer, max_length = 512)

In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 32
test_model(model_bin, balanced_small_dataset, tokenizer, batch_size, device)

Evaluating: 100%|██████████| 7/7 [00:13<00:00,  1.93s/it]

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.80      0.79      0.79       100
     Class 1       0.79      0.80      0.80       100

    accuracy                           0.80       200
   macro avg       0.80      0.80      0.79       200
weighted avg       0.80      0.80      0.79       200

Accuracy: 79.50%


('              precision    recall  f1-score   support\n\n     Class 0       0.80      0.79      0.79       100\n     Class 1       0.79      0.80      0.80       100\n\n    accuracy                           0.80       200\n   macro avg       0.80      0.80      0.79       200\nweighted avg       0.80      0.80      0.79       200\n',
 0.795)

In [22]:
summary_test_dataset = CustomDataset(summary_dataset, tokenizer, max_length = 512)

In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 32
test_model(model_bin, summary_test_dataset, tokenizer, batch_size, device)

Evaluating: 100%|██████████| 4/4 [00:08<00:00,  2.19s/it]

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.54      0.74      0.62        50
     Class 1       0.58      0.36      0.44        50

    accuracy                           0.55       100
   macro avg       0.56      0.55      0.53       100
weighted avg       0.56      0.55      0.53       100

Accuracy: 55.00%


('              precision    recall  f1-score   support\n\n     Class 0       0.54      0.74      0.62        50\n     Class 1       0.58      0.36      0.44        50\n\n    accuracy                           0.55       100\n   macro avg       0.56      0.55      0.53       100\nweighted avg       0.56      0.55      0.53       100\n',
 0.55)

## Testing lightning roberta

In [ ]:
pip install pytorch_lightning

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl

class BinaryClassifierPL(pl.LightningModule):
    def __init__(self, model, lr=2e-5):
        super().__init__()
        self.save_hyperparameters()
        self.model = model
        self.lr = lr
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids=input_ids, attention_mask=attention_mask)

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['binary_label'].long()

        outputs = self(input_ids, attention_mask)
        logits = outputs.logits
        loss = self.criterion(logits, labels)

        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['binary_label'].long()

        outputs = self(input_ids, attention_mask)
        logits = outputs.logits
        loss = self.criterion(logits, labels)

        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()

        # Log loss and accuracy averaged across the epoch
        self.log("val_loss", loss, prog_bar=True, on_epoch=True, on_step=False)
        self.log("val_acc", acc, prog_bar=True, on_epoch=True, on_step=False)

        return {"val_loss": loss, "val_acc": acc}

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr)

In [ ]:
from transformers import RobertaForSequenceClassification
import torch

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

state_dict = torch.load("/content/drive/MyDrive/hallucinations/lightning/model_roberta_5_epochs.pth")

# Filter out keys for just the roberta model
new_state_dict = {k.replace("model.model.", ""): v for k, v in state_dict.items() if k.startswith("model.model.")}

roberta = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)
roberta.load_state_dict(new_state_dict)

lightning_model = BinaryClassifierPL(roberta)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm
import torch

def test_model(model, test_dataset, tokenizer, batch_size, device, collate_fn):
    # Create DataLoader for the test dataset
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

    model.to(device)
    model.eval()

    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['binary_label'].to(device).long()

            # Forward pass using model's defined forward()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits if hasattr(outputs, 'logits') else outputs  # handle LightningModule

            probabilities = torch.softmax(logits, dim=1)[:, 1]  # Get probs for class 1
            predictions = (probabilities >= 0.5).long()

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    report = classification_report(all_labels, all_predictions, target_names=["Class 0", "Class 1"], zero_division=1)
    accuracy = accuracy_score(all_labels, all_predictions)

    print("\nClassification Report:")
    print(report)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    return report, accuracy


In [ ]:
balanced_small_dataset = CustomDataset(small_balanced_df, tokenizer, max_length = 512)

In [ ]:
summary_dataset_test = CustomDataset(summary_dataset, tokenizer, max_length = 512)

In [ ]:
test_model(lightning_model, balanced_small_dataset, tokenizer, 16, "cuda", collate_fn=collate_fn)

Evaluating: 100%|██████████| 13/13 [00:10<00:00,  1.25it/s]


Classification Report:
              precision    recall  f1-score   support

     Class 0       0.90      0.90      0.90       100
     Class 1       0.90      0.90      0.90       100

    accuracy                           0.90       200
   macro avg       0.90      0.90      0.90       200
weighted avg       0.90      0.90      0.90       200

Accuracy: 90.00%


('              precision    recall  f1-score   support\n\n     Class 0       0.90      0.90      0.90       100\n     Class 1       0.90      0.90      0.90       100\n\n    accuracy                           0.90       200\n   macro avg       0.90      0.90      0.90       200\nweighted avg       0.90      0.90      0.90       200\n',
 0.9)

In [ ]:
test_model(lightning_model, summary_dataset_test, tokenizer, 16, "cuda", collate_fn=collate_fn)

Evaluating: 100%|██████████| 7/7 [00:08<00:00,  1.16s/it]


Classification Report:
              precision    recall  f1-score   support

     Class 0       0.57      0.88      0.69        50
     Class 1       0.74      0.34      0.47        50

    accuracy                           0.61       100
   macro avg       0.66      0.61      0.58       100
weighted avg       0.66      0.61      0.58       100

Accuracy: 61.00%


('              precision    recall  f1-score   support\n\n     Class 0       0.57      0.88      0.69        50\n     Class 1       0.74      0.34      0.47        50\n\n    accuracy                           0.61       100\n   macro avg       0.66      0.61      0.58       100\nweighted avg       0.66      0.61      0.58       100\n',
 0.61)